In [1]:
#installation
# !conda install pytorch torchvision cudatoolkit=10.2 -c pytorch
# !pip install transformers
# !pip install pytorch_transformers

In [2]:
# Tutorial copied from https://github.com/huggingface/transformers#usage

In [3]:
import torch
from torch import nn
from transformers import BertModel, BertTokenizer

class PretrainedModel(nn.Module):
    def __init__(self):
        super(PretrainedModel, self).__init__()
        self.model = BertModel.from_pretrained(
            'bert-base-uncased',
            output_hidden_states=True,
            output_attentions=True
        ).cuda()
        self.model.eval()
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    def forward(self, text):
        input_ids = torch.tensor([self.tokenizer.encode(text, add_special_tokens=True)]).cuda()
        all_hidden_states, all_attentions = self.model(input_ids)[-2:]
        return all_hidden_states
        
model = PretrainedModel()
hidden_states = model("Here is some text to encode")

In [4]:
len(hidden_states)

13

In [5]:
class TinyBert(nn.Module):
    def __init__(self):
        super(TinyBert, self).__init__()
        
    def forward(self, text):
        return

In [6]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.pretrained_model = PretrainedModel()
        self.tinybert = TinyBert()

    def forward(self, text):
        pretrained_hidden = self.pretrained_model(text)
        

In [7]:
mdl = Model()

In [8]:
from torch.utils.data import Dataset
from itertools import islice
from torch.utils.data import DataLoader

class CustomDataset(Dataset):
    def __init__(self, filename, num_bunches = 100):
        self.num_bunches = num_bunches
        self.num_lines = 114180969
        self.bunch_width = self.num_lines // num_bunches
        self.filename = filename
        self.set_bunch(0)
    def set_bunch(self, bunch_idx):
        start = bunch_idx * self.bunch_width
        end = (bunch_idx + 1) * self.bunch_width
        with open('enwiki-latest-pages-articles_preprocessed.txt', encoding='iso-8859-1') as f:
            lines = [line[:-1] for line in islice(f, start, end)]
        self.X = lines
    def preprocess(self, text):
        return text
    def __len__(self):
        return len(self.X)
    def __getitem__(self, index):
        return self.X[index]

In [9]:
dataset = CustomDataset('enwiki-latest-pages-articles_preprocessed.txt')

#Wrap it around a dataloader
dataloader = DataLoader(dataset, batch_size = 64, num_workers = 0)

class MaskLMDataset:
    def __init__(self, dataset, dataloader):
        self.dataset = dataset
        self.dataloader = dataloader

In [10]:
# dataset.set_bunch(10)

In [11]:
itr = 0
for x in dataloader:
    print(len(x))
    itr += 1
    if itr > 2:
        break

64
64
64


In [12]:
x[0]

'Anarchist tactics have shifted during the course of the last century.'